### 方針
 ※ 使用データ...頂いた小麦のデータ
1. 適切な塩基の組み合わせを選ぶ（特徴量選択）
2. １で選んだ転写因子の組み合わせを選んだときにもっともエネルギーが低くなるような関数を設定する
3. 表現型(収穫量)の要素を一つずつ変化させる。この前後でエネルギーがどのくらい変わったのか確認する（比）。
4. 乱数(0 ≦ R ≦ 1の一様乱数)より高かったら採択、低かったら棄却
5. ３に戻る
6. ４に100回たどり着くたびにlistに記録


### 仮定
- 小麦データの収量行列は１行４列。
- 候補として最大値、最長値、平均値を考えたが、今回は0列目を使うことにする（よう言われました）

<br/>


### 1. 特徴量選択
(考え)変数間の相互作用を考えたほうが良いのだろう、反復特徴量選択は時間がかかる→モデルベース特徴量選択 <br/>
(結論)1279塩基から640塩基が残った

In [1]:
# 説明変数
import pandas as pd
data_SNP = pd.read_table('CIMMYTwheat_markers.txt', header=None,  sep =  ' ')
print('(SNPサンプル数, 塩基数)=', data_SNP.shape)

(SNPサンプル数, 塩基数)= (599, 1279)


In [2]:
# 目的変数
data_yields = pd.read_table('CIMMYTwheat_yields.txt', header=None,  sep =  ' ')
print('SNPサンプル数：', len(data_yields))
print(type(data_yields))
print(data_yields.shape)
print(data_yields.iloc[0]) #0行目
print(data_yields.iloc[0, 0]) #[0, 0]

'''
data_amount_list = []
for i in range(len(data_yields)):
    canditate = data_yields.iloc[i, 0]
    for j in range(1, 4):
        if data_yields.iloc[i, j] >= canditate:
            canditate = data_yields.iloc[i, j]
    data_amount_list.append(canditate)
'''

data_amount_list = []
for i in range(len(data_yields)):
    data_amount_list.append(data_yields.iloc[i, 0])

#print(data_amount)
print(len(data_amount_list))
print(type(data_amount_list))
print(data_amount_list[3])

SNPサンプル数： 599
<class 'pandas.core.frame.DataFrame'>
(599, 4)
0    1.671629
1   -1.727470
2   -1.890285
3    0.050916
Name: 0, dtype: float64
1.671629
599
<class 'list'>
0.7854395000000001


In [3]:
data_amount = pd.Series(data_amount_list)

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_SNP, data_amount, test_size=0.3) #x, y ,test0.3, train0.7

In [5]:
print(y_train.ndim)
print(y_train.shape)

print(y_test.ndim)
print(y_test.shape)

1
(419,)
1
(180,)


In [13]:
# Lasso回帰
import numpy as np
from sklearn.linear_model import Lasso
lasso = Lasso().fit(x_train, y_train)
print("Training set score: {:.2f}".format(lasso.score(x_train, y_train)))
print("Test set score: {:.2f}".format(lasso.score(x_test, y_test)))
print("Number of features used:", np.sum(lasso.coef_ != 0))

Training set score: 0.00
Test set score: -0.00
Number of features used: 0


In [6]:
# モデルベース特徴量選択（embedded method、組み込み法）
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor

# estimator として RandomForestRegressor を使用。重要度が median 以上のものを選択
select = SelectFromModel(RandomForestRegressor(n_estimators = 100, random_state = 42), threshold = 'median')
select.fit(x_train, y_train)

#yはselect.fitしなくていい
x_train_selected = select.transform(x_train)
x_test_selected = select.transform(x_test)

print('x_train.shape: {}'.format(x_train.shape))
print('x_train_l1_shape: {}'.format(x_train_selected.shape))

mask = select.get_support()
print(mask)
print(type(mask))

x_train.shape: (419, 1279)
x_train_l1_shape: (419, 640)
[ True False  True ...  True  True False]
<class 'numpy.ndarray'>


In [10]:
mask = select.get_support()
print(mask)
print(type(mask))
print(len(mask)) #もとの特徴量の数

[ True  True  True ...  True  True  True]
<class 'numpy.ndarray'>
1279


In [7]:
from sklearn import preprocessing, utils

lab_enc = preprocessing.LabelEncoder()

train_scores_encoded = lab_enc.fit_transform(y_train)
print(utils.multiclass.type_of_target(train_scores_encoded))

test_scores_encoded = lab_enc.fit_transform(y_test)
print(utils.multiclass.type_of_target(test_scores_encoded))

multiclass
multiclass


コード確認しました。
まずLogisticRegressionの学習のほう
train_scores_encoded, test_scores_encodedというのは、それぞれy_train, y_testの中の順位をとったものになっていますね (例えば、y_train = [42.0, 1.0, 3.141, 2.718, 0.0]の場合、train_scores_encoded = [4, 1, 3, 2, 0])。予測モデルは与えられた入力だけから出力を予測するので、全体のデータセットを知らないのに順位を予測しなさいといってもできません。つまりスコアも低くなります。
予測モデルを作るのであれば、LinearRegression等にしてx_trainからy_trainを直接予測するのがよいと思います。

In [8]:
#線形回帰（最小二乗法）
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(x_train_selected, y_train)
print("Training set score: {:.2f}".format(lr.score(x_train_selected, y_train)))
print("Test set score: {:.2f}".format(lr.score(x_test_selected, y_test)))
# 過剰適合

Training set score: 1.00
Test set score: -2.23


In [9]:
# リッジ回帰(デフォルトで、alpha=1.0)
from sklearn.linear_model import Ridge
ridge = Ridge().fit(x_train_selected, y_train)
print("Training set score: {:.2f}".format(ridge.score(x_train_selected, y_train)))
print("Test set score: {:.2f}".format(ridge.score(x_test_selected, y_test)))
# まだ過剰適合

# 係数とは w のこと
# Ridgeモデルでは、モデルの簡潔さ（絶対値が0に近い係数の数）と訓練セットに対する性能がトレードオフ
# alphaを増やすと、係数はより0に近くなり、訓練セットに対する性能は低下するが、汎化にはそのほうが良い可能性もある

Training set score: 0.98
Test set score: -0.71


In [10]:
# リッジ回帰(alpha=10)
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 10).fit(x_train_selected, y_train)
print("Training set score: {:.2f}".format(ridge.score(x_train_selected, y_train)))
print("Test set score: {:.2f}".format(ridge.score(x_test_selected, y_test)))
# alpha=10にしたにもかかわらずテストスコアが低すぎる

Training set score: 0.88
Test set score: 0.01


In [11]:
# リッジ回帰
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 20).fit(x_train_selected, y_train)
print("Training set score: {:.2f}".format(ridge.score(x_train_selected, y_train)))
print("Test set score: {:.2f}".format(ridge.score(x_test_selected, y_test)))
# まだ過剰適合

Training set score: 0.83
Test set score: 0.10


(学習に関してわかったこと)<br/>
- （特徴量選択＋）線形回帰では表せない（教師あり学習、線形回帰）
- lasso回帰でしてもスコアでない
- 今回は0列目を使ったが、別の値を使ったら精度が上がるかもしれない

In [8]:
'''
#モデルベース特徴量選択を用いた結果
from sklearn.linear_model import LogisticRegression
score = LogisticRegression().fit(x_train_selected, train_scores_encoded).score(x_test_selected, test_scores_encoded)
print(f"x_train_selected.shape : {x_train_selected.shape}")
#print(f'Score with only selected features : {score:.3f}')


#score = LogisticRegression().fit(x_train_l1, y_train).score(x_test_selected, y_test)
#print("Test score: {:.3f}".format(score))  # 0と出る
'''

/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


x_train_selected.shape : (419, 640)


###  確率場

- 確率場$Z$がギブス的である ⇔ そのときに確率密度関数が👇となっている
$$
f(z) =  \frac{e^{-E(z)}}{Z}
$$
<br/>
- エネルギー関数$E$をもつギブス分布（or ボルツマン分布）= 👆に対応して定まる分布
- $Z$は定数


<br/>

### 2. エネルギー関数の設定
- $E()$の設定→元祖maskが一番小さくなるように、相互情報量ではmask同士が一番大きくなる
- 確率をどう書くか→$f(z)$の設定、定数は無視
- エネルギーの大きさ比較→比を計算
- 乱数発生→uniform...
- 採択、棄却→更新するか否かの決断、エネルギーが一つ前のより小さかったら採択

In [ ]:
E = [] #Eは配列、どんどん入れていく(一つ後ろで計算の際使用される)
def Energy(mask_trial):
    return 1-normalized_mutual_info_score(mask_trial, mask) #エネルギーの定義≡1-標準化相互情報量 とした
#np.append(E, Energy(mask)) #第一要素を追加
E.append(Energy(mask))
#print(Energy(mask))
#print(mask)
print(type(E))
print(E[0])


#mask_trial ...new_yによって選ばれた塩基mask(x)、forループを回すたび値が変わる、maskと比較するためにあるだけなので保存する必要なし

save_y = [] #ほしいベクトルを貯める配列
new_y = data_amount #以降要素を一つずつ変えていく #data_amountはSeries

import math

for i in range(1, 100):   
    new_y[i] = 1 - new_y[i] #要素一つだけ変えた
    
    select = SelectFromModel(RandomForestRegressor(n_estimators = 100, random_state = 42), threshold = 'median', max_features=640)
    select.fit(data_SNP, new_y)
    data_SNP_l1 = select.transform(data_SNP)
    mask_trial = select.get_support()

    #np.append(E, Energy(mask_trial))  #mask_trialは一つのndarrayの中で順番に要素を更新していく
    E.append(Energy(mask_trial))
    r = math.exp(-E[i])/math.exp(-E[i-1])
    R = np.random.uniform(0.0, 1.0)
    if R >= r : #f(z)が一個前のものよりも小さかったら⇔E[i]が一個前のものよりも大きかったら
        new_y[i] =1 - new_y[i] #元に戻す   
    if i%100 == 0:
        save_y.append(new_y)
    else:
            continue
    if len(save_y) > 1:
        break